In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym.envs.registration import register
import time
import os

In [2]:
# env = gym.make('MountainCar-v0')
env = gym.make('MountainCar-v0', render_mode="human")
env.reset()
env.close()

In [3]:
# maxV = []
# maxP = []
# for i in range(10000):
#     env.render()
#     observation, reward, done, info, check = env.step(env.action_space.sample())
#     time.sleep(0.03)
#     maxP.append(observation[0])
#     maxV.append(observation[1])
#     if done:
#         env.reset()
#         print(np.max(maxP), np.max(maxV))
# env.close()

In [4]:
# print(np.max(maxP), np.max(maxV))

In [5]:
# print(np.min(maxP), np.min(maxV))

In [6]:
EPOCHS = 20000
ALPHA = 0.8
GAMMA = 0.9
NO_OF_BINS = 10

epsilon = 1.0
min_epsilon = 0.01
max_epsilon = 1.0
decay_val = 0.0045

In [7]:
def createBins(NO_OF_BINS):
    binPosition = np.linspace(-1.2, 0.6, NO_OF_BINS)
    binVelocity = np.linspace(-0.6, 0.6, NO_OF_BINS)

    return np.array([binPosition, binVelocity])

In [8]:
bins = createBins(NO_OF_BINS)


In [9]:
def discritizeObs(observation, bins):
    binnedObs = []
    for i, obs in enumerate(observation):
        discreetVal = np.digitize(obs, bins[i])
        binnedObs.append(discreetVal)

    return tuple(binnedObs)

In [16]:
qTableShape = (NO_OF_BINS, NO_OF_BINS, env.action_space.n)
qTable = np.zeros(qTableShape)
qTable[1]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [1]:
def egas(qTable, epsilon, discreetState):
    rn = np.random.random()
    if rn > epsilon:
        state_row = qTable[discreetState]

    else:
        action = env.action_space.sample()


SyntaxError: invalid syntax (3229517052.py, line 3)

In [18]:
a = np.zeros((3,5,2))
a

array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]])